# Extraction d'indicateurs de l'INSEE par API 

In [24]:
import pandas as pd
from lxml import etree 
import requests
import time 
import openpyxl 
import pickle 

In [25]:
def save_csv(lst,nom):    
    df = pd.DataFrame(lst)
    df.to_csv(f'/{nom}.csv')

In [26]:

chemin_fichier = 'data/dict/Dict_defaillance_identifiant.pickle'
with open(chemin_fichier, 'rb') as fichier:
    objet = pickle.load(fichier)
    
print(objet)


{"Nombre de défaillances d'entreprises par date de jugement - Cumul brut glissant sur 12 mois - Bourgogne-Franche-Comté - Agriculture, sylviculture et pêche": '001740071', "Nombre de défaillances d'entreprises par date de jugement - Cumul brut glissant sur 12 mois - Bourgogne-Franche-Comté - Industrie": '001740072', "Nombre de défaillances d'entreprises par date de jugement - Cumul brut glissant sur 12 mois - Bretagne - Agriculture, sylviculture et pêche": '001655999', "Nombre de défaillances d'entreprises par date de jugement - Cumul brut glissant sur 12 mois - Bretagne - Industrie": '001656000', "Nombre de défaillances d'entreprises par date de jugement - Cumul brut glissant sur 12 mois - Centre-Val de Loire - Agriculture, sylviculture et pêche": '001656429', "Nombre de défaillances d'entreprises par date de jugement - Cumul brut glissant sur 12 mois - Centre-Val de Loire - Industrie": '001656430', "Nombre de défaillances d'entreprises par date de jugement - Cumul brut glissant sur 1

In [27]:
with open('data/dict/Dict_defaillance_identifiant.pickle','rb') as pickle_file:
    content = pickle.load(pickle_file)
    
    liste_siren = []
    liste_ko_siren = []
    liste_nom_indicateurs = []
    
    for cle, valeur in content.items():
            if valeur[0:2] != '00': 
                liste_ko_siren.append(cle)
                
            else :
                 liste_nom_indicateurs.append(cle)
                 liste_siren.append(valeur)

In [28]:
for e in liste_siren:
    print(e)

001740071
001740072
001655999
001656000
001656429
001656430
001656501
001656534
001740108
001740097
001656238
001740083
001740084
001740120
001740131
001740142
001656429
001655952
001656453
001656454
001740143
001740154
001740074
001740077
001656002
001656001
001655954
001656431
001656534
001656503
001740097
001740098
001740097
001656240
001656095
001740086
001740122
001740120
001740142
001740142
001655954
001655953
001656456
001656455
001740146
001740145
001740076
001656004
001740111
001656434
001656433
001656507
001656534
001656098
001740120
001655956
001655955
001656458
001656457
001740148
001740147
001740079
001656007
001656006
001656436
001656435
001656506
001656534
001740117
001740103
001740102
001656244
001656244
001740094
001740090
001740120
001740126
001740142
001740138
001655958
001656460
001656459
001740154
001740150
001740080
001740077
001656001
001656005
001656462
001656438
001656535
001656534
001740117
001740113
001740102
001656271
001656242
001740093
001740089
001740129


In [29]:
with open('data/dict/Dict_indices_prix_identifiant.pickle','rb') as pickle_file2:
    content2 = pickle.load(pickle_file2)
    liste_siren2    = []
    liste_ko_siren2 = []
    liste_nom_indicateurs2 = []
        
    for cle, valeur in content2.items():
            if (valeur[0:2] not in ('00','01','10')): 
                liste_ko_siren2.append(cle)
            else :
                liste_nom_indicateurs2.append(cle)
                liste_siren2.append(valeur)

In [31]:
print(liste_nom_indicateurs[1])

Nombre de défaillances d'entreprises par date de jugement - Cumul brut glissant sur 12 mois - Bourgogne-Franche-Comté - Industrie


### Extraction défaillances -> defaillances.xlsx

In [34]:
# Replace YOUR_API_KEY with your actual API key
api_key = '58dad6a7-7e27-30ee-9da8-b25c54b7888e'
# Set the headers with the API key
headers = {
    'Authorization': f"Bearer {api_key}",
    'Accept':'application/xml'
}

# Créer un nouveau classeur Excel
workbook = openpyxl.Workbook()

for siren_number,title in zip(liste_siren,liste_nom_indicateurs):
    data = []
    url = f"https://api.insee.fr/series/BDM/V1/data/SERIES_BDM/{siren_number}"  

    time.sleep(3)

    # Make a GET request to the INSEE API
    response = requests.get(url, headers=headers)
    resultat = response.content 
    root = etree.XML(resultat) 
    try:
        for elem in root[1]:
            # Parcourir le dataset
            for obs in elem :
            # Récupérer les attributs de chaque observation
                attributes = obs.attrib
                attributes["OBS_VALUE"] = attributes["OBS_VALUE"]
                data.append(attributes)

        # Créer un dataframe à partir de la liste de données
        df_response = pd.DataFrame(data)
    except IndexError:
        print(f"Erreur d'index pour le SIREN {siren_number}. Le traitement sera ignoré.")
        liste_siren.remove(siren_number)
        liste_ko_siren.append(siren_number)

    # Créer un nouvel onglet pour chaque indicateur
    worksheet = workbook.create_sheet(title=siren_number)
    
    title_row = 1
    worksheet.cell(row=title_row, column=8, value=title)  # Placer le titre dans la colonne H
    worksheet.merge_cells(start_row=title_row, start_column=8, end_row=title_row, end_column=17)  # Fusionner les cellules de H à Q
    worksheet.cell(row=title_row, column=8).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)  # Activer le renvoi à la ligne automatique
    # Écrire les en-têtes de colonne dans la première ligne de l'onglet
    header_row = 2
    for col_idx, column_name in enumerate(df_response.columns, start=1):
        cell = worksheet.cell(row=header_row, column=col_idx)
        cell.value = column_name

    # Écrire les données du dataframe dans les lignes suivantes
    data_rows = df_response.to_numpy().tolist()
    start_row = header_row + 1
    for row_idx, row_data in enumerate(data_rows, start=start_row):
        for col_idx, cell_value in enumerate(row_data, start=1):
            cell = worksheet.cell(row=row_idx, column=col_idx)
            cell.value = cell_value

# Supprimer la feuille par défaut
workbook.remove(workbook["Sheet"])

# Enregistrer le classeur Excel avec le nom "données_api.xlsx"
output_file = ("data/result/insee/defaillances_api.xlsx")
workbook.save(output_file)

# Confirmer que le fichier a été créé
print(f"Le fichier {output_file} a été généré avec succès.")

Le fichier data/result/insee/defaillances_api.xlsx a été généré avec succès.


In [30]:
# on récupère les siren pour lesquels il n'y a pas d'informations 
save_csv(liste_ko_siren,"liste_ko_1")
save_csv(liste_ko_siren2,"liste_ko_2")

### Extraction pour le fichier 2 
indicateurs_prix.xlsx

In [35]:
import requests
import time
import pandas as pd
import openpyxl
from lxml import etree
import sys

api_key = '58dad6a7-7e27-30ee-9da8-b25c54b7888e'
headers = {
    'Authorization': f"Bearer {api_key}",
    'Accept': 'application/xml'
}

def make_request(siren_number, headers):
    """Effectuer une requête GET vers l'API INSEE et retourner le contenu XML."""
    try:
        url = f"https://api.insee.fr/series/BDM/V1/data/SERIES_BDM/{siren_number}"
        time.sleep(3)  # limit rate
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # raise an error for status 4xx and 5xx
        return etree.XML(response.content)
    except requests.exceptions.HTTPError as errh:
        if errh.response.status_code == 401:
            print("Erreur HTTP 401 : Non autorisé. Vérifiez les informations d'authentification.")
            sys.exit(1)
        else:
            print(f"Erreur HTTP: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Erreur de Connexion: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Timeout: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"Erreur lors de la requête: {err}")
    return None

def xml_to_dataframe(root):
    """Transformer le contenu XML en DataFrame."""
    data = []
    try:
        if root is not None:
            for elem in root[1]:
                for obs in elem:
                    attributes = obs.attrib
                    data.append(attributes)
            return pd.DataFrame(data)
    except Exception as e:
        print(f"Erreur lors du traitement du XML: {e}")
    return pd.DataFrame()

def create_sheet(workbook, df, siren_number, title):
    """Créer et remplir une feuille dans un classeur Excel."""
    try:
        if not df.empty:
            worksheet = workbook.create_sheet(title=siren_number)
            config_sheet(worksheet, df, title)
        else:
            print(f"Pas de données pour {siren_number}")
    except Exception as e:
        print(f"Erreur lors de la création de la feuille Excel pour {siren_number}: {e}")

def config_sheet(worksheet, df, title):
    """Configurer les titres et en-têtes de la feuille Excel."""
    try:
        worksheet.cell(row=1, column=8, value=title).alignment = openpyxl.styles.Alignment(horizontal='center', vertical='center', wrap_text=True)
        worksheet.merge_cells(start_row=1, start_column=8, end_row=1, end_column=17)

        for col_idx, column_name in enumerate(df.columns, start=1):
            worksheet.cell(row=2, column=col_idx, value=column_name)

        for row_idx, row_data in enumerate(df.to_numpy().tolist(), start=3):
            for col_idx, cell_value in enumerate(row_data, start=1):
                worksheet.cell(row=row_idx, column=col_idx, value=cell_value)

    except Exception as e:
        print(f"Erreur lors de la configuration de la feuille: {e}")

def main(liste_siren, liste_noms):
    """Fonction principale pour la création du fichier excel."""
    workbook = openpyxl.Workbook()
    liste_ko_siren = []  # Liste pour garder une trace des SIREN pour lesquels la requête a échoué

    for siren_number, title in zip(liste_siren, liste_noms):
        xml_data = make_request(siren_number, headers)

        if xml_data is not None:
            df = xml_to_dataframe(xml_data)
            
            if not df.empty:
                create_sheet(workbook, df, siren_number, title)
            else:
                print(f"Aucune donnée trouvée pour {siren_number}. Ajout à la liste des échecs.")
                liste_ko_siren.append(siren_number)
        else:
            liste_ko_siren.append(siren_number)

    if "Sheet" in workbook.sheetnames:
        workbook.remove(workbook["Sheet"])  # Supprimer la feuille par défaut si elle existe encore

    output_file = "indicateurs_prix.xlsx"
    workbook.save(output_file)
    print(f"Le fichier {output_file} a été généré avec succès.")

    if liste_ko_siren:
        print("Les requêtes pour les SIREN suivants ont échoué ou n'ont retourné aucune donnée :")
        for siren in liste_ko_siren:
            print(siren)


main(liste_siren2, liste_nom_indicateurs2)



Erreur HTTP: 404 Client Error:  for url: https://api.insee.fr/series/BDM/V1/data/SERIES_BDM/010539854
Erreur HTTP: 404 Client Error:  for url: https://api.insee.fr/series/BDM/V1/data/SERIES_BDM/010539490
Erreur HTTP: 404 Client Error:  for url: https://api.insee.fr/series/BDM/V1/data/SERIES_BDM/109940795
Le fichier indicateurs_prix.xlsx a été généré avec succès.
Les requêtes pour les SIREN suivants ont échoué ou n'ont retourné aucune donnée :
010539854
010539490
109940795


# XSLSX Writter dans Pandas

https://xlsxwriter.readthedocs.io/working_with_pandas.html

In [37]:
import xlsxwriter 
df = pd.DataFrame({
    'Categories': ['Catégorie 1', 'Catégorie 2', 'Catégorie 3'],
    'Valeurs': [10, 20, 30]
})


with pd.ExcelWriter('fichier_spécifique.xlsx', engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='Feuille1', index=False)

   
    workbook = writer.book
    worksheet = writer.sheets['Feuille1']

    
    worksheet.write('E1', 'Titre du document')

    # Ajouter un graphique à la feuille
    chart = workbook.add_chart({'type': 'column'})

    # Sélection des données
    chart.add_series({
        'name':       'Valeurs',
        'categories': '=Feuille1!$A$2:$A$4',
        'values':     '=Feuille1!$B$2:$B$4',
    })

    # Style du graphique
    chart.set_title({'name': 'Exemple de Graphique'})
    chart.set_x_axis({'name': 'Catégories'})
    chart.set_y_axis({'name': 'Valeurs'})

    
    worksheet.insert_chart('D2', chart)